### Setup

```bash
sudo docker run \
    -it --rm \
    -v /Users/chi/Documents/GitHub:/tf \
    -p 8888:8888 \
    -p 6006:6006 \
    --ipc="host" \
    tensorflow/tensorflow:nightly-py3-jupyter

-it -d --rm
tensorflow/tensorflow:2.0.0rc0-py3-jupyter
```

@Deprecated
```bash
sudo docker build .
sudo docker run 
    --runtime=nvidia -d -it \
    --name=maskrcnn \
    -v=$(pwd):/notebooks \
    -p=8888:8888 \
    --ipc="host" \
    maskrcnn-benchmark
sudo docker logs maskrcnn
sudo docker start maskrcnn
sudo docker exec -it maskrcnn /bin/bash
```

## Notes

model-free vs model-based: model包含actor & critic，在critic方面，model-based有含renderer net, model-free沒有

In [4]:
# !apt update
# !apt -y install git
# !pip3 install torch==1.2.0+cpu torchvision==0.4.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
# !pip3 install ray[rllib] ray[debug] psutil pandas

In [1]:
# !pip install ray 
# ray[rllib] ray[debug] pandas

%cd /notebooks/LearningToPaint/rllib
!python test_env.py

/notebooks/LearningToPaint/rllib
W0921 08:23:09.028197 140114280478464 deprecation.py:323] From /miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
2019-09-21 08:23:09,622	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-09-21_08-23-09_619946_3334/logs.
2019-09-21 08:23:09,736	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:64887 to respond...
2019-09-21 08:23:09,873	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:26618 to respond...
2019-09-21 08:23:09,876	INFO services.py:809 -- Starting Redis shard with 2.52 GB max memory.
2019-09-21 08:23:09,941	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-09-21_08-23-09_619946_3334/logs.


2019-09-21 08:23:17,260	WARNING util.py:145 -- The `start_trial` operation took 0.1033315658569336 seconds to complete, which may be a performance bottleneck.
(pid=3362) WARNING: Logging before flag parsing goes to stderr.
(pid=3362) W0921 08:23:21.242904 140683541686016 deprecation.py:323] From /miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=3362) Instructions for updating:
(pid=3362) non-resource variables are not supported in the long term
(pid=3362) 2019-09-21 08:23:21,915	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
(pid=3362) 2019-09-21 08:23:21.915724: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=3362) 2019-09-21 08:23:21.924720: I tensorfl

^C
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/miniconda/envs/py36/lib/python3.6/site-packages/ray/node.py", line 594, in _kill_process_type
    process.wait()
  File "/miniconda/envs/py36/lib/python3.6/subprocess.py", line 1457, in wait
    (pid, sts) = self._try_wait(0)
  File "/miniconda/envs/py36/lib/python3.6/subprocess.py", line 1404, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt


In [8]:
# install openai baselines and other required packages
# !pip install gym tensorflow

# !apt-get update && apt-get -y install cmake libopenmpi-dev python3-dev zlib1g-dev
# %cd /notebooks/baselines
# # !git clone https://github.com/openai/baselines.git  # use github-desktop to clone repository
# !pip install -e .

# # install local gym-canvas as package
# %cd /notebooks/LearningToPaint/gym-canvas
# !pip install -e .

## Training

### train renderer

In [52]:
%cd /notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail
!python main.py --env-name "gym_canvas:canvas-v0" --num-processes 1 --algo ppo --use-gae --lr 2.5e-4 --clip-param 0.1 --value-loss-coef 0.5 --num-steps 5 --num-mini-batch 4 --log-interval 1 --use-linear-lr-decay --entropy-coef 0.01

# !python baseline/train_renderer.py
# !tensorboard --logdir train_log --port=6006

/notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail
Tuple
Traceback (most recent call last):
  File "main.py", line 254, in <module>
    main()
  File "main.py", line 55, in main
    base_kwargs={"recurrent": args.recurrent_policy},
  File "/notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail/a2c_ppo_acktr/model.py", line 41, in __init__
    raise NotImplementedError
NotImplementedError


### train RL

* make data available

In [10]:
!python baseline/train.py --max_step=4 --env_batch=1 --warmup=1 --validate_interval=1 --debug

mkdir: cannot create directory './model': File exists
Renderer model loaded.
observation_space (1, 128, 128, 7) action_space 13
 #0: steps:4 interval_time:0.22 train_time:24.56
evaluating
 Step_0000007: mean_reward:0.288 mean_dist:0.201 var_dist:0.000
 #1: steps:8 interval_time:2.69 train_time:34.23
evaluating
 Step_0000011: mean_reward:-8.902 mean_dist:0.619 var_dist:0.000
 #2: steps:12 interval_time:2.55 train_time:43.24
evaluating
 Step_0000015: mean_reward:-7.753 mean_dist:0.417 var_dist:0.000
 #3: steps:16 interval_time:2.68 train_time:55.46
evaluating
 Step_0000019: mean_reward:-1.631 mean_dist:0.326 var_dist:0.000
^C


### Supervised training

In [1]:
!python baseline/train_supervised.py

Renderer model loaded.
step 0: loss 9167.975586
^C
Traceback (most recent call last):
  File "baseline/train_supervised.py", line 84, in <module>
    train(args.batch_size, args.episode_steps)